In [9]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.ssa.*
import jline.solvers.mva.*
import jline.util.matrix.Matrix

GlobalConstants.setVerbose(VerboseLevel.STD)

In [10]:
val model = Network("model")
val n = 5  // number of items
val m = 2  // cache capacity

val delay = Delay(model, "Delay")
val cacheNode = Cache(model, "Cache", n, m, ReplacementStrategy.FIFO)
val jobClass = ClosedClass(model, "JobClass", 1, delay, 0)
val hitClass = ClosedClass(model, "HitClass", 0, delay, 0)
val missClass = ClosedClass(model, "MissClass", 0, delay, 0)

delay.setService(jobClass, Exp(1.0))

// Uniform item references
val probArray = Matrix(1, n).fill(1.0 / n)
val pAccess = DiscreteSampler(probArray)
cacheNode.setRead(jobClass, pAccess)

cacheNode.setHitClass(jobClass, hitClass)
cacheNode.setMissClass(jobClass, missClass)
val P = model.initRoutingMatrix()
P.set(jobClass, jobClass, delay, cacheNode, 1.0)
P.set(hitClass, jobClass, cacheNode, delay, 1.0)
P.set(missClass, jobClass, cacheNode, delay, 1.0)

model.link(P)

In [11]:
import jline.solvers.AvgTable
import jline.solvers.Solver

val solver = arrayOfNulls<Solver>(3)
val avgTable = arrayOfNulls<AvgTable>(3)

In [12]:
solver[0] = CTMC(model, "keep", false, "seed", 1)
avgTable[0] = (solver[0] as CTMC).avgNodeTable
avgTable[0]!!.print()

CTMC analysis [method: default, lang: java, env: 17.0.15] completed in 0.008906s.
Station            JobClass   QLen        Util        RespT       ResidT      ArvR        Tput        
------------------------------------------------------------------------------------------------------
Delay              JobClass   1.00000     1.00000     1.00000     0.33333     0.33333     0.33333     
Delay              HitClass   0.00000     0.00000     0.00000     0.00000     0.33333     0.33333     
Delay              MissClass  0.00000     0.00000     0.00000     0.00000     0.33333     0.33333     
Cache              HitClass   0.00000     0.00000     0.00000     0.00000     0.00000     0.33333     
Cache              MissClass  0.00000     0.00000     0.00000     0.00000     0.00000     0.33333     
CS_Cache_to_Delay  JobClass   0.00000     0.00000     0.00000     0.00000     0.33333     1.00000     
CS_Cache_to_Delay  HitClass   0.00000     0.00000     0.00000     0.00000     0.33333     0.00

In [13]:
model.reset()

In [14]:
solver[1] = SSA(model, "samples", 10000, "verbose", true, "method", "serial", "seed", 1)
avgTable[1] = (solver[1] as SSA).avgNodeTable
avgTable[1]!!.print()

SSA samples:   10000
SSA analysis [method: serial, lang: java, env: 17.0.15] completed in 0.979958s. Iterations: 10001.
Station            JobClass   QLen        Util        RespT       ResidT      ArvR        Tput        
------------------------------------------------------------------------------------------------------
Delay              JobClass   1.00000     1.00000     1.00000     1.00000     1.00000     1.00000     
Cache              JobClass   0.00000     0.00000     0.00000     0.00000     1.00000     0.00000     
Cache              HitClass   0.00000     0.00000     0.00000     0.00000     0.00000     0.39997     
Cache              MissClass  0.00000     0.00000     0.00000     0.00000     0.00000     0.60003     
CS_Cache_to_Delay  JobClass   0.00000     0.00000     0.00000     0.00000     0.00000     1.00000     
CS_Cache_to_Delay  HitClass   0.00000     0.00000     0.00000     0.00000     0.39997     0.00000     
CS_Cache_to_Delay  MissClass  0.00000     0.00000     0.

In [ ]:
model.reset()

In [16]:
// Additional MVA run
model.reset()
val additionalSolver = MVA(model, "seed", 1)
println("\nSOLVER: MVA")
val avgTable3 = additionalSolver.avgNodeTable
avgTable3.print()


SOLVER: MVA
MVA analysis [method: default(exact), lang: java, env: 17.0.15] completed in 0.005317s. Iterations: 2.
Station            JobClass   QLen        Util        RespT       ResidT      ArvR        Tput        
------------------------------------------------------------------------------------------------------
Delay              JobClass   1.00000     1.00000     1.00000     1.00000     1.00000     1.00000     
Cache              JobClass   0.00000     0.00000     0.00000     0.00000     1.00000     0.00000     
Cache              HitClass   0.00000     0.00000     0.00000     0.00000     0.00000     0.40000     
Cache              MissClass  0.00000     0.00000     0.00000     0.00000     0.00000     0.60000     
CS_Cache_to_Delay  JobClass   0.00000     0.00000     0.00000     0.00000     0.00000     1.00000     
CS_Cache_to_Delay  HitClass   0.00000     0.00000     0.00000     0.00000     0.40000     0.00000     
CS_Cache_to_Delay  MissClass  0.00000     0.00000     0.0000